In [ ]:
from ultralytics import YOLO
import torch
from PIL import Image
from torchvision.transforms import v2
import torchvision.transforms as T
import glob
import os
import posixpath
import cv2


In [ ]:
model = YOLO("yolov8n.yaml") 
model = YOLO("checkpoints/ckp_yolo_model.pt")  

source = sorted(glob.glob('./datasample/original/images/test/*.jpg'),
                    key = lambda i: (len(posixpath.basename(i)), i))
print(source)
result = model.predict(source, save=True, conf=0.5)

In [ ]:
print(len(result))
i = range(0,len(result))


In [ ]:
bbox1 = []
name = []

for n in i:
    boxes = result[n].boxes.xywhn.tolist()
    classes = result[n].boxes.cls.tolist()
    names = result[n].names
    confidences = result[n].boxes.conf.tolist()
    # k = result[n].boxes.keypoints.tolist()
    bbox1.append(boxes)
    name.append(confidences)


In [ ]:
bbox= []

for n in i:
    b = bbox1[n]
    c = b[0]
    bbox.append(c)
    

In [ ]:
def add_margin(pil_img, top, right, bottom, left, color):
    width, height = pil_img.size
    new_width = width + right + left
    new_height = height + top + bottom
    result = Image.new(pil_img.mode, (new_width, new_height), color)
    result.paste(pil_img, (left, bottom))
    return result

In [ ]:
from pathlib import Path
Path("./output/yolo_output/").mkdir(parents=True, exist_ok=True)

for x, image in zip(bbox, source):
    im = Image.open(image)
    tens = cv2.imread(image) 
    image_width = tens.shape[1]
    image_height = tens.shape[0] 

    x_center = x[0]
    y_center = x[1]
    width = x[2]
    height = x[3]

    w = width * image_width
    h = height * image_height
    end_point0 = int((x_center * image_width) - (w/2))
    end_point1 = int((y_center * image_height) - (h/2))
    start_point0 = int(w + end_point0)
    start_point1 = int(h + end_point1)
 
    cropped_image1 = tens[end_point1:start_point1, end_point0:start_point0]
    cropped = Image.fromarray(cropped_image1)
    resize = v2.Resize(size=(1024, 512)) 
    cropped_image2 = resize(cropped)
    im_new = add_margin(cropped_image2, 0, 256, 0, 256, (0, 0, 0))

    base, ext = os.path.splitext(os.path.basename(image))  
    ext = ext.lower()
    im_new.save('./output/yolo_output/'f"{base}{ext}")
